## 元编程
当使用点号访问属性(或者是 调用`getattr` `setattr`)失败时候，调用 `__getattr__` 和 `__setattr__`来计算属性

In [2]:
class test:
    a = 233
    def __getattr__(self, name):
        return name
t = test()
print(t.a, t.xxx)

233 xxx


with语句可以使用两个上下文管理器

构建实例的特殊方法是 `__new__` 是一个类方法

对`__dict__`操作添加一堆属性

In [5]:
class test:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
d = {'a':1,'b':2}
t = test(**d)
print(t.a,t.b)

1 2


**特性会覆盖实例属性** <br>
特性都是类属性，但是特性管理的其实是实例属性的读取 <br>
实例属性遮盖类的数据属性,实例属性不会遮盖类特性 <br>
`obj.attr` 这样的表达式不会从`obj`开始寻找`attr` 而是从`obj.__class__`开始 而且仅当类中没有名为`attr`的特性时，才会在对象实例中寻找。 <br>
**这条规则不仅适用于特性 还适用于一整类描述符 覆盖型描述符**

In [29]:
class test:
    
    def __init__(self, w):
        self.w = w
    
    @property
    def w(self):
        return self.__w
    
    @w.setter
    def w(self, v):
        if v > 0:
            self.__w = v
        else:
            raise ValueError('w must > 0')

### 处理属性的重要属性和函数

In [36]:
t = test(1)

**影响属性处理方式的特殊属性**

1 `__class__`对象所属类的引用 `obj.__class__`与`type(obj)`作用相同

2 `__dict__`一个映射 存储对象或类的可写属性

3 `__slots__`类可以定义这个属性限制实例能有哪些属性。是一个字符串组成的元组，指明允许有的属性 如果没有`__dict__`那么该类的实例就没有`__dict__`属性

**处理属性的特殊方法** <br>
**假定特殊方法是从类中获取**
使用点号或者内置的`getattr` `hasattr` `setattr`函数存取属性都会出发下述列表中相应的特殊方法，但是通过`__dict__`属性进行读写属性时候 不会触发这些特殊方法，因此可以使用这种方法跳过这些特殊方法, 假设一个类`Class`的实例对象为`obj`

1 `__delattr__(self, name)` 只要使用del语句删除属性就会调用这个方法 `ex`: `del obj.attr`语法触发`Class.__delattr(obj, 'attr)`

2 `__getattr__`仅当获取属性失败，已经搜索过`obj Class`和超类之后调用

3 `obj.attr`或`getattr(obj, 'attr', 42)`都会触发`Class.__getattribute__(obj, 'attr')` 当尝试获取指定属性时调用 当寻找的属性是特殊属性或特殊方法时除外。

重写该方法容易出现无限递归的错误

4 `__setattr__`设定属性值